# + Requirements

1. List of strategies on AllocateSmartly
2. CSVs of each strategy
3. Update every month (at rebalance date)
4. Import data table for each strategy on PortfolioVisualizer as "shares" and benchmarks
5. Update strategy (strategy of several strategies)
6.

In [3]:
import os
from selenium import webdriver
import pandas as pd
import datetime, time, csv

# Allocate Smartly

In [4]:
# prepare Data folders

import shutil

try:
    shutil.rmtree(b'/home/scubamut/projects/pvautomate/Data')
except:
    pass

os.makedirs(b'/home/scubamut/projects/pvautomate/Data')
os.makedirs(b'/home/scubamut/projects/pvautomate/Data/strategy_returns')
os.makedirs(b'/home/scubamut/projects/pvautomate/Data/benchmark_returns')

## Login to allocatesmartly.com and generate CSV files for the Strategies


In [5]:
data_path = '/home/scubamut/projects/pvautomate/Data/'

browser = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")
# browser = webdriver.Firefox()
browser.get('https://allocatesmartly.com/login')
browser.set_window_position(1,1)
browser.maximize_window()
time.sleep(5)
user = browser.find_element_by_id('user_login')
user.send_keys(os.environ.get('ASNAME'))
password = browser.find_element_by_id('user_pass')
password.send_keys(os.environ.get('ASPASSWD'))
time.sleep(5)
login = browser.find_element_by_id('wp-submit')
login.click()

browser.get('https://allocatesmartly.com/members/strategies/')

strategies = pd.read_html(browser.page_source, header=0)[0]
strategies = strategies.filter(items=[c for c in strategies.columns][1:-2])
# remove Strategy 'Trading ...'
strategies.Strategy = strategies.Strategy.apply(lambda x: x[:x.find('Trading')])
strategies[38:]

,Strategy,AnnualReturn (20Y),SharpeRatio (20Y)
38,Davis' Three Way Model,8.7%,0.64
39,60/40 Benchmark,6.5%,0.54
40,Faber's Sector Relative Strength (Sector RS),7.4%,0.47
41,Faber's Ivy Portfolio,6.9%,0.47


In [6]:
# for each strategy, get monthly table and generate CSV files 

for s in strategies.Strategy:
    print(s)
    browser.find_elements_by_link_text(s)[0].click()
    browser.find_element_by_id('sx-periodic-table')
    table = pd.read_html(browser.page_source, header=1)[2]
    table.columns = ['Year','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','Total']
    # need to change 60/40 to 60_40 for 
    table.to_csv(data_path + '/strategy_returns/' + s.replace('/','_')+'.csv')
    browser.execute_script("window.history.go(-1)")
    time.sleep(2)
    
browser.close()

Adaptive Asset Allocation
Protective Asset Allocation
Allocate Smartly's Meta Strategy
Vigilant Asset Allocation
Protective Asset Allocation - CPR
Keuning's Generalized Protective Momentum
Varadi's Percentile Channels
US Risk Parity Trend Following
Flexible Asset Allocation
Stoken's Active Combined Asset (ACA) - Monthly
Classical Asset Allocation - Defensive
Stoken's Active Combined Asset (ACA)
Efficiente Index
Elastic Asset Allocation - Defensive
Robust Asset Allocation - Balanced
Faber's Global Tactical Asset Alloc. - Agg. 6
Elastic Asset Allocation - Offensive
Global Risk Parity Trend Following
Varadi's Minimum Correlation Portfolio
Faber's Trinity Portfolio Lite
Antonacci's Composite Dual Momentum
Faber's Global Tactical Asset Alloc. 13 (GTAA 13)
Faber's Global Tactical Asset Alloc. 5 (GTAA 5)
Tactical Permanent Portfolio
US Max Diversification
Faber's Global Tactical Asset Alloc. - Agg. 3
Dalio's All-Weather Portfolio
Classical Asset Allocation - Offensive
PortfolioCharts' Golden 

In [7]:
# 
strats = strategies.copy()
strats.to_csv(data_path + 'strats.csv')

## Generate symbols (benchmark names) for use by Portfolio Visualizer

In [8]:
data_path = '/home/scubamut/projects/pvautomate/Data/'
strategies = pd.read_csv(data_path + 'strats.csv')

In [9]:
#  new Stratnames column and remove brackets
strategies['Stratnames'] = pd.Series(strategies.Strategy).str.replace(r"\(.*\)","")
# rename 60/40 Benchmark to B6040 Benchmark
strategies['Stratnames'] = strategies.Stratnames.str.replace('40', 'B6040')
# remove developer from name
strategies['Stratnames'] = strategies.Stratnames.str.replace("/('\w+)|(\w+'\w+)|(\w+')|(\w+)/", "")
# new column for Developers
strategies['Developer'] = strategies.Strategy.str.extract("/('\w+)|(\w+'\w+)|(\w+')|(\w+)/", expand=True)[1]
# create unique name >= 6 characters for Portfolio Visualizer Bencmarks
strategies['Name'] = [''.join([c for c in s if c.isupper()]) for s in strategies.Stratnames]
strategies['Numbers'] = strategies.Stratnames.str.extract('(\d+)', expand=True).fillna('')
strategies['Symbol'] = strategies['Name'] + strategies['Numbers']
# # append 0s so that Share Name is at least 6 characters
strategies['Symbol'] = [s+'00000'[:6-len(s)]for s in strategies.Symbol]
strategies['Symbol'] = strategies['Symbol'].str.replace('4000', 'B6040')
strategies = strategies.filter(items=['Strategy', 'Stratnames', 'Symbol', 'AnnualReturn (20Y)', 'SharpeRatio (20Y)'])
strategies['Strategy'] = strategies['Strategy'].str.replace('/','_')
strategies[38:]

,Strategy,Stratnames,Symbol,AnnualReturn (20Y),SharpeRatio (20Y)
38,Davis' Three Way Model,Three Way Model,TWM000,8.7%,0.64
39,60_40 Benchmark,B6040 Benchmark,BB6040,6.5%,0.54
40,Faber's Sector Relative Strength (Sector RS),Sector Relative Strength,SRS000,7.4%,0.47
41,Faber's Ivy Portfolio,Ivy Portfolio,IP0000,6.9%,0.47


In [10]:
# verfy that all Securities are unique

len(strategies) == len(strategies.Symbol.unique())

True

In [12]:
strategies.to_csv('/home/scubamut/projects/pvautomate/Data/ASstrategies.csv', index=None)
strategies[:5]

,Strategy,Stratnames,Symbol,AnnualReturn (20Y),SharpeRatio (20Y)
0,Adaptive Asset Allocation,Adaptive Asset Allocation,AAA000,13.1%,1.18
1,Protective Asset Allocation,Protective Asset Allocation,PAA000,10.1%,1.05
2,Allocate Smartly's Meta Strategy,Allocate Meta Strategy,AMS000,8.9%,1.05
3,Vigilant Asset Allocation,Vigilant Asset Allocation,VAA000,14.4%,1.05
4,Protective Asset Allocation - CPR,Protective Asset Allocation - CPR,PAACPR,9.6%,1.01


## Generate Benchmark Returns for use by Portfolio Visualizer 

In [13]:
def update_strategy_data(data_path, strategy, symbol):
    """
    For each strategy, save latest data table from website, 
    generate csv data to save benchmark with format suitable for
    PortfolioVisualizer
    
    - data_path : windows folder name for data
    - symbol : strategy Symbol 
    - strategy : strategy Name
    
    """
    df = pd.read_csv(data_path + 'strategy_returns/' + strategy + '.csv',index_col=[0] ).fillna('0.0%')

    df1 = pd.DataFrame(columns=['Period','Return'])
    df1 = df1.append({'Period': 'Period', 'Return': 'Return'}, ignore_index=True)
    for row in range(0,len(df)):
        for column in range(1,13):
            year, month, value = df.iloc[row,0], column, df.iloc[row, column]
#             period = (datetime.date (year, month, 1) - datetime.timedelta (days = 1)).strftime('%#m/%#d/%Y')
            next_month = datetime.date (year, month, 1).replace(day=28) + datetime.timedelta(days=4)  # this will never fail
            period = next_month - datetime.timedelta(days=next_month.day)
            newline = str(period) + u',' + value
            df1 = df1.append({'Period': period, 'Return': value}, ignore_index=True)

    df1.to_csv(data_path + 'benchmark_returns/' + symbol.replace('/','_')+'.csv', index=False, header=False, quoting=csv.QUOTE_NONNUMERIC) 

In [14]:
data_path = '/home/scubamut/projects/pvautomate/Data/'

for n in range(len(strategies)):
# for n in [1]:
    symbol = strategies.Symbol[n]
    strategy = strategies.Strategy[n]
    print((symbol, strategy))
    update_strategy_data(data_path, strategy, symbol)
    
#browser.close()

('AAA000', 'Adaptive Asset Allocation')
('PAA000', 'Protective Asset Allocation')
('AMS000', "Allocate Smartly's Meta Strategy")
('VAA000', 'Vigilant Asset Allocation')
('PAACPR', 'Protective Asset Allocation - CPR')
('GPM000', "Keuning's Generalized Protective Momentum")
('PC0000', "Varadi's Percentile Channels")
('USRPTF', 'US Risk Parity Trend Following')
('FAA000', 'Flexible Asset Allocation')
('ACAM00', "Stoken's Active Combined Asset (ACA) - Monthly")
('CAAD00', 'Classical Asset Allocation - Defensive')
('ACA000', "Stoken's Active Combined Asset (ACA)")
('EI0000', 'Efficiente Index')
('EAAD00', 'Elastic Asset Allocation - Defensive')
('RAAB00', 'Robust Asset Allocation - Balanced')
('GTAAA6', "Faber's Global Tactical Asset Alloc. - Agg. 6")
('EAAO00', 'Elastic Asset Allocation - Offensive')
('GRPTF0', 'Global Risk Parity Trend Following')
('MCP000', "Varadi's Minimum Correlation Portfolio")
('TPL000', "Faber's Trinity Portfolio Lite")
('CDM000', "Antonacci's Composite Dual Moment

# Portfolio Visualizer

## Login to PortfolioVisualizer

In [16]:
from selenium import webdriver
import time

browser = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")
# browser = webdriver.Firefox()
browser.set_window_position(1,1)
# browser.set_window_size(1038, 875)
browser.maximize_window()
browser.get('https://www.portfoliovisualizer.com/login')
time.sleep(2)
user = browser.find_element_by_id('username')
user.send_keys(os.environ.get('PVNAME'))
password = browser.find_element_by_id('password')
password.send_keys(os.environ.get('PVPASSWD'))
time.sleep(2)
login = browser.find_element_by_id('submitButton')
login.click()

## Navigate to Import Benchmarks

In [17]:
len(strategies)

42

In [18]:
strategies[-3:]

,Strategy,Stratnames,Symbol,AnnualReturn (20Y),SharpeRatio (20Y)
39,60_40 Benchmark,B6040 Benchmark,BB6040,6.5%,0.54
40,Faber's Sector Relative Strength (Sector RS),Sector Relative Strength,SRS000,7.4%,0.47
41,Faber's Ivy Portfolio,Ivy Portfolio,IP0000,6.9%,0.47


In [19]:
data_path = '/home/scubamut/projects/pvautomate/Data/benchmark_returns/'

browser.get('https://www.portfoliovisualizer.com/preferences#import')

for n in range(len(strategies)):
# for n in [1]:
    
    symbol = strategies.Symbol[n]
    strategy = strategies.Strategy[n]
    print (symbol, strategy)
    
    browser.maximize_window()
    
    browser.find_element_by_id("benchmarkName").clear()
    browser.find_element_by_id("benchmarkName").send_keys(strategy)
    
    browser.find_element_by_id("upload").clear()
    browser.find_element_by_id("upload").send_keys(data_path + symbol +'.csv')
    
    browser.find_element_by_id('percentageValues_chosen').click()
    browser.find_element_by_xpath('//*[@id="percentageValues_chosen"]/div/ul/li[2]').click()

    browser.find_element_by_id("benchmarkSymbol").clear()
    browser.find_element_by_id("benchmarkSymbol").send_keys(symbol)

    browser.find_element_by_id('benchmarkAssetClass_chosen').click()
#     browser.find_element_by_xpath('//*[@id="benchmarkAssetClass_chosen"]/div/ul/li[2]').click()
    browser.find_element_by_css_selector('#benchmarkAssetClass_chosen > div > ul > li:nth-child(2)').click()
    browser.implicitly_wait(90)
    browser.find_element_by_id("importBenchmarkButton").click()

    browser.execute_script("window.history.go(-1)")
    time.sleep(2)
    
browser.close()

AAA000 Adaptive Asset Allocation
PAA000 Protective Asset Allocation
AMS000 Allocate Smartly's Meta Strategy
VAA000 Vigilant Asset Allocation
PAACPR Protective Asset Allocation - CPR
GPM000 Keuning's Generalized Protective Momentum
PC0000 Varadi's Percentile Channels
USRPTF US Risk Parity Trend Following
FAA000 Flexible Asset Allocation
ACAM00 Stoken's Active Combined Asset (ACA) - Monthly
CAAD00 Classical Asset Allocation - Defensive
ACA000 Stoken's Active Combined Asset (ACA)
EI0000 Efficiente Index
EAAD00 Elastic Asset Allocation - Defensive
RAAB00 Robust Asset Allocation - Balanced
GTAAA6 Faber's Global Tactical Asset Alloc. - Agg. 6
EAAO00 Elastic Asset Allocation - Offensive
GRPTF0 Global Risk Parity Trend Following
MCP000 Varadi's Minimum Correlation Portfolio
TPL000 Faber's Trinity Portfolio Lite
CDM000 Antonacci's Composite Dual Momentum
GTAA13 Faber's Global Tactical Asset Alloc. 13 (GTAA 13)
GTAA50 Faber's Global Tactical Asset Alloc. 5 (GTAA 5)
TPP000 Tactical Permanent Port

In [ ]:
# REMEMBER THAT THE LAST MONTH MAY BE INCOMPLETE!!